<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/HubSpot_Logo.svg/220px-HubSpot_Logo.svg.png" alt="drawing" width="200" align='left'/>

# Hubspot - Update linkedinbio
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Hubspot/Hubspot_create_association.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import library

In [ ]:
from naas_drivers import hubspot
import naas
import pandas as pd
from googlesearch import search
import time
import re

### Enter Hubspot api key

In [ ]:
auth_token = "YOUR_HUBSPOT_API_KEY"

### Connect to hubspot

In [ ]:
hs = hubspot.connect(auth_token)

### Schedule your notebook everyday

In [ ]:
naas.scheduler.add(cron="0 6 * * *")

### Get all contacts in Hubspot

In [ ]:
properties_list = [
    "hs_object_id",
    "firstname",
    "lastname",
    "linkedinbio",
]
hubspot_contacts = hs.contacts.get_all(properties_list).fillna("Not Defined")
hubspot_contacts

# Model

### Filter to get "Not Defined" linkedinbio and defined "firstname" and "lastname"

In [ ]:
df_to_update = hubspot_contacts.copy()

# Filter on "Not defined"
df_to_update = df_to_update[(df_to_update.firstname != "Not Defined") & 
                            (df_to_update.lastname != "Not Defined") &
                            (df_to_update.linkedinbio == "Not Defined")].reset_index(drop=True)

df_to_update

### Search bio in Google with firstname and lastname

In [ ]:
def get_bio(firstname, lastname):
    # Init linkedinbio
    linkedinbio = None
    
    # Create query
    query = f"{firstname}+{lastname}+Linkedin"
    print("Google query: ", query)
    
    # Search in Google
    for i in search(query, tld="com", num=10, stop=10, pause=2):
        pattern = "https:\/\/.+.linkedin.com\/in\/.([^?])+"
        result = re.search(pattern, i)

        # Return value if result is not None
        if result != None:
            linkedinbio = result.group(0).replace(" ", "")
            return linkedinbio
        else:
            time.sleep(2)
    return linkedinbio

In [ ]:
for _, row in df_to_update.iterrows():
    firstname = row.firstname
    lastname = row.lastname
    
    # Get linkedinbio
    linkedinbio = get_bio(firstname, lastname)
    df_to_update.loc[_, "linkedinbio"] = linkedinbio
    
df_to_update

# Output

### Update Linkedin bio in Hubspot

In [ ]:
for _, row in df_to_update.iterrows():
    # Init data
    data = {}
    
    # Get data
    hs_object_id = row.hs_object_id
    linkedinbio = row.linkedinbio

    # Update LK Bio
    if linkedinbio != None:
        data = {"properties": {"linkedinbio": linkedinbio}}
    hs.contacts.patch(hs_object_id, data)